In [ ]:
# Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer
import pandas as pd
import gc

# !pip install crucio
# !pip install category_encoders
from category_encoders import *
from crucio import MTDF, SMOTE, ADASYN, ICOTE, MWMOTE, TKRKNN
dirThis = '/content/drive/MyDrive/1經濟學/專題/'

In [ ]:
class dataLoader() :
  def __init__(self, dt, path = '/content/drive/MyDrive/1經濟學/專題/featureNew/', dd = 0):
    dirThis = '/content/drive/MyDrive/1經濟學/專題/'
    if(dd == 1) :
      self.data = pd.read_csv(path + 'use.csv')
    else :
      self.data = dt
    
    self.trainHistory = pd.read_csv(dirThis + 'trainHistory.csv')
    self.testHistory = pd.read_csv(dirThis + 'testHistory.csv')

    codl = LabelEncoder()
    self.target = pd.DataFrame(codl.fit_transform(self.trainHistory['repeater']), 
                 columns = ['repeater'])
    self.targetCont = self.trainHistory['repeattrips']

    id = self.data['id']
    self.data.drop(['id', 'offerdate'], axis = 1, inplace = True)
    self.train = self.data[id.isin(self.trainHistory['id'])].reset_index(drop = True)
    self.test = self.data[id.isin(self.testHistory['id'])].reset_index(drop = True)
    self.stdCols = self.data.columns[4:]

    self.encol = ['market', 'offer', 'chain', 'offervalue']


  def Encoder(self, encode) :
    if(encode == 'ordinal') :
      encoder = OrdinalEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)   
    elif(encode == 'onehot') :
      encoder = OneHotEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'target') :
      encoder = TargetEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'binary') :
      encoder = BinaryEncoder(cols = self.encol).fit(self.trainSet)
    elif(encode == 'cat') :
      encoder = CatBoostEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'woe') :
      encoder = WOEEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'helmert') :
      encoder = HelmertEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'leave') :
      encoder = LeaveOneOutEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    elif(encode == 'hash') :
      encoder = HashingEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)
    else :
      encoder = OrdinalEncoder(cols = self.encol).fit(self.trainSet, self.targetSet)

    print("encoder : " + encode)
    self.trainSet = encoder.transform(self.trainSet)
    self.testSet = encoder.transform(self.testSet)

  def Standardization(self, st) :
    if(st == 'std') :
      scaler = StandardScaler()
    elif(st == 'minmax') :
      scaler = MinMaxScaler()
    elif(st == 'abs') :
      scaler = MaxAbsScaler()
    elif(st == 'robust') :
      scaler = RobustScaler()
    elif(st == 'power') :
      scaler = PowerTransformer()
    elif(st == 'qnormal') :
      scaler = QuantileTransformer(output_distribution = 'normal')
    elif(st == 'quantile') :
      scaler = QuantileTransformer()
    else :
      scaler = StandardScaler()
    
    print("standardization : " + st)
    tmp = self.trainSet[self.stdCols]
    self.trainSet[self.stdCols] = pd.DataFrame(scaler.fit_transform(tmp), columns = self.stdCols)

    tmp = self.testSet[self.stdCols]
    self.testSet[self.stdCols] = pd.DataFrame(scaler.fit_transform(tmp), columns = self.stdCols)

  def Oversampling(self, over) :
    if(over == 'smote') :
      method = SMOTE() 
    elif(over == 'adasyn') :
      method = ADASYN()
    elif(over == 'icote') :
      method = ICOTE()
    elif(over == 'mwmote') :
      method = MWMOTE()
    elif(over == 'tkrknn') :
      method = TKRKNN()
    else :
      method = MTDF()
    
    print("oversampling : " + over)
    newTrain = method.balance(self.trainSet.join(self.targetSet), 'repeater').reset_index(drop = True)
    
    self.trainSet = newTrain['repeater']
    self.targetSet = newTrain.drop(['repeater'], axis = 1)

  def getData(self) :
    return self.data
  def getTarget(self):
    return self.target
  def getTargetCont(self) :
    return self.targetCont
  def getTrain(self) :
    return self.train
  def getTest(self) :
    return self.test
  def getSet(self, encode = 'ordinal', st = 'none', over = 'none') :
    self.trainSet = self.train.copy()
    self.testSet = self.test.copy()
    self.targetSet = self.target.copy()
    if(encode != 'none') :
      self.Encoder(encode)
    if(st != 'none') :
      self.Standardization(st)
    if(over != 'none') :
      self.Oversampling(over)

    return self.trainSet, self.testSet, self.targetSet

In [ ]:
path = '/content/drive/MyDrive/1經濟學/專題/featureNew/'
dt = pd.read_csv(path + 'use.csv')

In [ ]:
dat = dt.copy()
newdata = dataLoader(dat)

In [ ]:
train = newdata.getTrain()
test = newdata.getTest()
target = newdata.getTarget()
data = newdata.getData()

In [ ]:
train, test, target = newdata.getSet(encode = 'onehot', st = 'std', over = 'none')

encoder : onehot
standardization : std


In [ ]:
print(train.shape)
print(target.shape)

(160057, 183)
(160057, 1)
